<a href="https://colab.research.google.com/github/camulro/Aprendizaje-II/blob/sesi%C3%B3n1/04_Pytorch_con_GPU_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![IDAL](https://i.imgur.com/tIKXIG1.jpg)  

#<strong>**Máster en Inteligencia Artificial Avanzada y Aplicada  IA^3**</strong>
---


# Que es CUDA?

Mucha gente confunde CUDA con un lengaje o con una API. No lo es. Es más que eso. CUDA en una plataforma de cálculo computerizado paralelo y un modelo de programación que permite aprovechar las GPUs para tareas de propósito general de una forma fácil y elegante. Los desarrolladores pueden continuar trabajando en C, C++, Fortran, Python y una lista cada día más amplia e incorpora extensiones de estos lenguajes en forma de unas pocas palabras clave básicas.

Estas palabras clave permiten al desarrollador expresar cantidades masivas de paralelismo y dirigir al compilador a la porción de la aplicación que se mapea a la GPU. En definitiva, hace que el acceso a la gran potencia computacional de las GPUs se haya incorporado en los lenguajes de programación de propósito general, permitiendo una gran expansión de técnicas y tecnologías que requieren de esa potencia, como las técnicas de aprensdizaje máquina, inteligencia artificial y más concretamente aprendizaje profundo (_deep learning_)

# Como instalo PyTorch para GPU?

En primer lugar es necesario tener una tarjeta gráfica NVIDIA compatible y con los drivers CUDA instalado y actualizados correctamente.  A continuación selecciona la versión de Pytorch correspondiente al descargarlo de la [página oficial](https://pytorch.org/get-started/locally/)

# Como saber si tienes CUDA disponible

In [1]:
import torch
torch.cuda.is_available()
# True

True

# Usando GPU y CUDA


In [2]:
## Id del dispositivo por defecto
torch.cuda.current_device()

0

In [3]:
# 0
torch.cuda.get_device_name(0) # Obtenemos el nombre del dispositivo ID '0'

'Tesla K80'

In [4]:
# Retorna el uso de memoria actual provocado por
# tensores en bytes para el dispositivo dado
torch.cuda.memory_allocated()

0

In [5]:
# Retorna la memoria gestionada por el gestor de memoria 
# en bytes para el dispositivo dado
torch.cuda.memory_reserved()

0

# Usando CUDA en lugar de CPU

In [6]:
# CPU
a = torch.FloatTensor([1.,2.])

In [7]:
a

tensor([1., 2.])

In [8]:
a.device

device(type='cpu')

In [10]:
# GPU
a = torch.FloatTensor([1., 2.]).cuda() #lo genera en gpu directamente


In [11]:
a.device

device(type='cuda', index=0)

In [12]:
torch.cuda.memory_allocated()

512

## Enviando modelos a la GPU

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
class Model(nn.Module): 
# Regresion logística
    def __init__(self, input_size=4, num_classes=3):
        super().__init__()
        self.linear = nn.Linear(input_size, num_classes)
        
    def forward(self, xb):
        out = self.linear(xb)
        return out

In [15]:
torch.manual_seed(32)
model = Model()

In [16]:
# Comprobación: discuss.pytorch.org/t/how-to-check-if-model-is-on-cuda
next(model.parameters()).is_cuda #comprobamos si los parámetros del modelo estan en cuda

False

In [17]:
gpumodel = model.cuda()

In [18]:
next(gpumodel.parameters()).is_cuda

True

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Datos/iris.csv')
X = df.drop('target',axis=1).values
y = df['target'].values


In [23]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [24]:
df.shape


(150, 5)

## Conjuntos Train-Test

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

## Convertir Tensores a .cuda() tensors

In [26]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

## Preparacion de datos

In [27]:
trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

## Función de coste, optimizador y evaluador

In [28]:
criterion = nn.CrossEntropyLoss() #función de pérdida, clasificar entre 3 clases de planta -> entropia cruzada
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [29]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds)) #devuelve el porcentaje total de predicciones acertadas

## Entrenamiento con GPU

In [31]:
import time #para medir el tiempo
epochs = 300
losses = []
accs =[]
start = time.time() #tiempo inicial
for i in range(epochs):
    i+=1
    y_pred = gpumodel.forward(X_train) #calcula regresión lineal
    loss = criterion(y_pred, y_train) #perdidas
    acc = accuracy(y_pred, y_train) #precisión
    losses.append(loss)
    accs.append(acc)
    
    # log:
    if i%10 == 1: #en los múltiplos de 10 imprime:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}  acc: {acc.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 0.26590708  acc: 0.96666664
epoch: 11  loss: 0.25990745  acc: 0.96666664
epoch: 21  loss: 0.25415474  acc: 0.96666664
epoch: 31  loss: 0.24863632  acc: 0.96666664
epoch: 41  loss: 0.24334060  acc: 0.96666664
epoch: 51  loss: 0.23825659  acc: 0.96666664
epoch: 61  loss: 0.23337373  acc: 0.96666664
epoch: 71  loss: 0.22868247  acc: 0.96666664
epoch: 81  loss: 0.22417334  acc: 0.96666664
epoch: 91  loss: 0.21983761  acc: 0.96666664
epoch: 101  loss: 0.21566698  acc: 0.96666664
epoch: 111  loss: 0.21165358  acc: 0.96666664
epoch: 121  loss: 0.20778990  acc: 0.96666664
epoch: 131  loss: 0.20406890  acc: 0.96666664
epoch: 141  loss: 0.20048389  acc: 0.96666664
epoch: 151  loss: 0.19702847  acc: 0.96666664
epoch: 161  loss: 0.19369666  acc: 0.96666664
epoch: 171  loss: 0.19048271  acc: 0.96666664
epoch: 181  loss: 0.18738127  acc: 0.96666664
epoch: 191  loss: 0.18438716  acc: 0.96666664
epoch: 201  loss: 0.18149552  acc: 0.96666664
epoch: 211  loss: 0.17870173  acc: 0.9666666

In [33]:
_, preds = torch.max(y_pred, dim=1) #y_pred devuelve probabilidades y predicciones, solo nos interesa preds
print(f'Aciertos: {torch.sum(preds == y_train).item()}')
print(f'Muestras totales: {len(preds)}')


Aciertos: 116
Muestras totales: 120


In [34]:
print(preds)

tensor([1, 1, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1,
        2, 2, 1, 2, 0, 0, 1, 0, 0, 1, 1, 1, 2, 2, 2, 2, 0, 0, 1, 1, 0, 1, 2, 1,
        2, 0, 2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 1, 0, 0, 2,
        1, 1, 0, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 0, 2, 1, 2, 1, 2, 1, 2, 0],
       device='cuda:0')


In [35]:
print(y_train)

tensor([1, 1, 2, 2, 2, 2, 2, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 2, 0, 0, 1, 2, 0, 1,
        2, 2, 1, 1, 0, 0, 2, 0, 0, 2, 1, 1, 2, 2, 2, 2, 0, 0, 1, 1, 0, 1, 2, 1,
        2, 0, 2, 0, 1, 0, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 0, 1, 0, 1, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 0, 0, 0, 0, 2, 2, 0, 1, 1, 2, 1, 0, 0, 1,
        1, 1, 0, 1, 1, 0, 2, 2, 2, 1, 2, 0, 1, 0, 0, 0, 2, 1, 2, 1, 2, 1, 2, 0],
       device='cuda:0')


# Curiosidad: Volviendo a CPU


In [36]:
torch.manual_seed(32)
model2 = Model()

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

trainloader = DataLoader(X_train, batch_size=60, shuffle=True)
testloader = DataLoader(X_test, batch_size=60, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

import time
epochs = 300
losses = []
start = time.time()
for i in range(epochs):
    i+=1
    y_pred = model2(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%10 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
print(f'TOTAL TRAINING TIME: {time.time()-start}')

epoch:  1  loss: 1.90774035
epoch: 11  loss: 1.90774035
epoch: 21  loss: 1.90774035
epoch: 31  loss: 1.90774035
epoch: 41  loss: 1.90774035
epoch: 51  loss: 1.90774035
epoch: 61  loss: 1.90774035
epoch: 71  loss: 1.90774035
epoch: 81  loss: 1.90774035
epoch: 91  loss: 1.90774035
epoch: 101  loss: 1.90774035
epoch: 111  loss: 1.90774035
epoch: 121  loss: 1.90774035
epoch: 131  loss: 1.90774035
epoch: 141  loss: 1.90774035
epoch: 151  loss: 1.90774035
epoch: 161  loss: 1.90774035
epoch: 171  loss: 1.90774035
epoch: 181  loss: 1.90774035
epoch: 191  loss: 1.90774035
epoch: 201  loss: 1.90774035
epoch: 211  loss: 1.90774035
epoch: 221  loss: 1.90774035
epoch: 231  loss: 1.90774035
epoch: 241  loss: 1.90774035
epoch: 251  loss: 1.90774035
epoch: 261  loss: 1.90774035
epoch: 271  loss: 1.90774035
epoch: 281  loss: 1.90774035
epoch: 291  loss: 1.90774035
TOTAL TRAINING TIME: 0.2914440631866455


## Fin del Notebook

Referencias y modelos empleados para el Notebook: 

*   Documentación de [Pytorch](https://pytorch.org/docs/stable/index.html) 
*   [PyTorch Tutorial for Deep Learning Researchers](https://github.com/yunjey/pytorch-tutorial) by Yunjey Choi
*   [FastAI](https://www.fast.ai/) development notebooks by Jeremy Howard.
*   Documentación y cursos en [Pierian Data](https://www.pieriandata.com/)
*   Tutoriales y notebooks del curso "Deep Learning with Pytorch: Zero to GANs" de [Aakash N S](https://jovian.ai/aakashns)
* [A visual proof that neural networks can compute any function](http://neuralnetworksanddeeplearning.com/chap4.html), también conocido como Teorema de Aproximación Universal
* [But what *is* a neural network?](https://www.youtube.com/watch?v=aircAruvnKk) - Una introducción muy intuitiva a lo que son las redes neuronales y lo que implican las capas ocultas.